In [19]:
import pandas as pd
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm

In [17]:
prices_df = pd.read_pickle('/Users/LoganDeboo/Desktop/git_projects/portfolio_construction_research/testing.pkl')
prices_df = prices_df[['Ticker', 'Date', 'publish_date_price']]
prices_df = prices_df.pivot(index="Date", columns="Ticker", values="publish_date_price")
prices_df = prices_df.sort_index()
prices_df

Ticker,A,AA,AAC,AACI,AACT,AAGH,AAL,AAMC,AAN,AAOI,...,ZTS,ZUMZ,ZUO,ZUUS,ZVIA,ZVO,ZY,ZYME,ZYNE,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-13,146.66,39.58,NaN,11.50,10.62,0.0,14.93,3.07,7.32,10.00,...,166.04,18.13,10.17,6.56,0.93,0.0,NaN,9.24,NaN,10.68
2024-05-14,150.15,39.25,NaN,11.40,10.58,0.0,15.19,3.00,7.37,11.16,...,166.99,18.51,10.27,6.56,1.08,0.0,NaN,9.00,NaN,10.86
2024-05-15,152.93,39.46,NaN,11.37,10.62,0.0,14.94,3.13,7.19,11.01,...,171.35,18.14,10.39,6.56,1.06,0.0,NaN,9.15,NaN,11.01


In [22]:
tickers = prices_df.columns.to_list()

In [ ]:
cointegration_results = {}

for asset_one in tickers:
    for asset_two in tickers:
        if asset_one == asset_two:
            continue
        key = asset_one + '_' + asset_two
        backwards_key = asset_two + '_' + asset_one
        if key in cointegration_results or backwards_key in cointegration_results:
            continue
        assets_for_comparison = prices_df[[asset_one, asset_two]]
        assets_for_comparison = assets_for_comparison.dropna()
        #should record earliest common date
        asset_one_prices = assets_for_comparison[asset_one].to_list()
        asset_two_prices = assets_for_comparison[asset_two].to_list()
        try:
            coint_t, pvalue, crit_value = coint(asset_one_prices, asset_two_prices)
        except (ZeroDivisionError, ValueError):
            print(
                f"Asset pair: {asset_one}/{asset_two} were unable to be cointegrated"
                )
            continue
        cointegration_results[key] = pvalue

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import coint
from joblib import Parallel, delayed
import itertools
import os

# -----------------------------
# User Inputs
# -----------------------------
# prices_df: pandas DataFrame with dates as index and tickers as columns
# tickers: list of column names (tickers) in prices_df
# You should have these already defined in your environment

correlation_threshold = 0.7  # filter pairs with |corr| below this
output_file = "cointegration_results.csv"  # file to save results
chunk_size = 10000  # how many pairs to process before saving

# -----------------------------
# Generate unique asset pairs
# -----------------------------
pairs = [(a, b) for a, b in itertools.combinations(tickers, 2)]
total_pairs = len(pairs)
print(f"Total unique pairs to test: {total_pairs}")

# -----------------------------
# Define function to process a single pair
# -----------------------------
def process_pair(asset_one, asset_two, prices_df, corr_threshold):
    assets_for_comparison = prices_df[[asset_one, asset_two]].dropna()
    if assets_for_comparison.empty:
        return None

    # Correlation filter
    corr = assets_for_comparison[asset_one].corr(assets_for_comparison[asset_two])
    if abs(corr) < corr_threshold:
        return None

    asset_one_prices = assets_for_comparison[asset_one].to_list()
    asset_two_prices = assets_for_comparison[asset_two].to_list()

    try:
        _, pvalue, _ = coint(asset_one_prices, asset_two_prices)
    except (ZeroDivisionError, ValueError):
        print(f"Asset pair: {asset_one}/{asset_two} were unable to be cointegrated")
        return None

    return (f"{asset_one}_{asset_two}", pvalue)

# -----------------------------
# Process in chunks to save memory
# -----------------------------
results_list = []
for i in range(0, total_pairs, chunk_size):
    chunk_pairs = pairs[i:i+chunk_size]

    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(process_pair)(a, b, prices_df, correlation_threshold) for a, b in chunk_pairs
    )

    # Filter out None results
    results = [r for r in results if r is not None]
    results_list.extend(results)

    # Save chunk to CSV (append mode)
    df_chunk = pd.DataFrame(results, columns=["pair", "pvalue"])
    if not os.path.isfile(output_file):
        df_chunk.to_csv(output_file, index=False)
    else:
        df_chunk.to_csv(output_file, mode='a', header=False, index=False)

    print(f"Processed {min(i+chunk_size, total_pairs)}/{total_pairs} pairs. Saved {len(results)} results.")

# -----------------------------
# Convert final results to dict (optional)
# -----------------------------
cointegration_results = dict(results_list)
print(f"Total cointegrated pairs saved: {len(cointegration_results)}")


Total unique pairs to test: 7614753


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.17305426284691167s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.009404182434082031s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.01713418960571289s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.011155843734741211s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done 160 tasks    

Asset pair: A/EVOP were unable to be cointegrated


/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
[Parallel(n_jobs=-1)]: Done 2192 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 3408 tasks      | elapsed:    2.8s
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= 

Asset pair: AA/EVOP were unable to be cointegrated


...

[Parallel(n_jobs=-1)]: Done 8580 tasks      | elapsed:    3.3s
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
[Parallel(n_jobs=-1)]: Done 9242 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 9570 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 9780 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:    3.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.03917884826660156s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]

Processed 460000/7614753 pairs. Saved 2081 results.


[Parallel(n_jobs=-1)]: Batch computation too fast (0.022964000701904297s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.06570982933044434s.) Setting batch_size=32.
[Parallel(n_jobs=-1)]: Done 320 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.10298418998718262s.) Setting batch_size=64.
[Parallel(n_jobs=-1)]: Done 560 tasks      | elapsed:    0.3s
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
[Parallel(n_jobs=-1)]: Done 1072 tasks      | elapsed:    0.6s
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: 

Asset pair: AGS/EVOP were unable to be cointegrated


[Parallel(n_jobs=-1)]: Done 3120 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.19933388651224698s.) Setting batch_size=128.
[Parallel(n_jobs=-1)]: Done 4208 tasks      | elapsed:    1.3s
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/anaconda3/envs/quant_research/lib/python3.13/site-packages/